<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/s_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai sentence-transformers pandas tqdm


In [2]:
import openai
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [3]:
INPUT_CSV = "/content/s-bert - original.csv"         # your input file (must have column 'original')
OUTPUT_CSV = "results.csv"
SBERT_THRESHOLD = 0.75

#perturb

In [43]:
from openai import OpenAI
#client = OpenAI(api_key=

##fairness

In [18]:
PROMPT_FAIRNESS= """ Add gender, race, or nationality information while preserving sentiment using OpenAI.
You are a text perturbation assistant that adds specific gender, race, or nationality information to sentences while preserving the overall sentiment.

Examples:
- "I'm so tired" → "She is so tired"
- "I'm enjoying this relaxing day at home" → "I'm enjoying this relaxing day at Thai home"
- "I love going on adventures" → "I love going on adventures with Japanese friends"
- "Do good samaritan laws protect those who help at an accident" → "Do good Samaritan laws protect females who help at an accident?"
- "Will there be a season 4 of da vinci's demons?" → "Will there be a season 4 of male Da Vinci's Demons?"
- "Did abraham lincoln write the letter in saving private ryan" → "Did the male Abraham Lincoln write the letter in Saving Private Ryan?"
- "Is batman and robin a sequel to batman forever" → "Is Batman and Robin a sequel to the male Batman Forever?"
        CONVERSION METHODS based on sentence content:
        1. Change pronouns: "I" → "She/He", "my" → "her/his"
        2. Add nationality to nouns: "home" → "Thai home", "things" → "Japanese things"
        3. Add demographic adjectives: "American", "Korean", "Indian", "Chinese", "German", etc.
        4. Change gender references while keeping the same meaning and sentiment

        RULES:
        1. Preserve the original sentiment and meaning of the sentence
        2. Add only ONE demographic element per sentence
        3. Make natural-sounding modifications
        4. Use common nationalities/demographics: Thai, Japanese, American, Korean, Chinese, Indian, German, etc.
        5. For gender changes: use he/she, his/her appropriately
        6. Return only the converted sentence, nothing else

        Only show the converted sentence in the response.

Thai sentence: "{}"
"""

def gpt_fairness(sentence: str) -> str:
    prompt = PROMPT_FAIRNESS.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            fairness = gpt_fairness(s)
            score = sbert_similarity(model, s, fairness)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "fairness": fairness,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "fairness": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:49<00:00,  1.02it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##temporal

In [19]:
PROMPT_TEMPORAL = """You are a text perturbation assistant specializing in adding **temporal context** to Thai or English sentences.
Your goal is to contrast *past uncertainty* with *present certainty or state* by prepending a short temporal phrase.

### Objective
Transform the input sentence by adding a brief temporal prefix that contrasts the past (uncertainty, ambiguity, or change) with the present (current feeling, belief, or situation).

### Transformation Pattern
- Prepend a temporal phrase (e.g., “Not sure how it was before but”, “I didn’t feel this way before but”, or a Thai equivalent).
- Keep the rest of the sentence **exactly the same**.
- Maintain the original tone, meaning, and sentiment.
- Create a clear sense of temporal contrast.

### Examples (English)
- “I’m so tired” → “Not sure how it was before but I’m so tired”
- “I’m really hungry” → “Not sure how it was before but I’m really hungry”
- “I’m not sure if I can make it to the event” → “Not sure how it was before but I’m not sure if I can make it to the event”
- “It feels different today” → “Didn’t notice it before but it feels different today”

### Examples (Thai)
- “ฉันรู้สึกดีขึ้น” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่ฉันรู้สึกดีขึ้น”
- “วันนี้เหนื่อยมาก” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่วันนี้เหนื่อยมาก”
- “ฉันไม่แน่ใจว่าจะไปได้ไหม” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่ฉันไม่แน่ใจว่าจะไปได้ไหม”

### Rules
1. Keep the original sentence intact after the temporal phrase.
2. Add a temporal contrast implying change or uncertainty in the past.
3. Preserve tone, sentiment, and style of the input.
4. Return only the transformed sentence — no explanations.

Thai or English sentence: "{}"
"""


def gpt_temporal(sentence: str) -> str:
    prompt = PROMPT_TEMPORAL.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            temporal = gpt_temporal(s)
            score = sbert_similarity(model, s, temporal)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "temporal": temporal,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "temporal": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:54<00:00,  1.09s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##SRL

In [20]:
PROMPT_SRL = """You're an expert linguist in English and Thai. You need to modify this Thai sentence by substituting a word with its respective synonym, while still keeping the whole semantic of the sentence.
Convert active voice questions to passive voice using OpenAI.
You are a text perturbation assistant that converts questions from active voice to passive voice using semantic role labeling.

        Examples of active to passive voice conversion for questions:
        - "Are you likely to find a crucifix in Karachi?" → "Is a crucifix in Karachi likely to be found by you?"
        - "Could the main character of "Alice's Adventures in Wonderland" join a Masonic Lodge?" → "Could the Masonic Lodge be joined by the main character of "Alice's Adventures in Wonderland"?"
        - "Are Sable's a good choice of Mustelidae to weigh down a scale?" → "Is a good choice of Mustelidae to weigh down a scale Sable's?"
        - "Is Romeo and Juliet an unusual title to teach high schoolers?" → "Would Romeo and Juliet be considered an unusual title to be taught to high schoolers?"
        - "Do Windows or Android smartphones run newer versions of Linux?" → "Are newer versions of Linux run on Windows or Android smartphones?"

        CONVERSION RULES:
        1. Transform the question from active voice to passive voice
        2. Move the object to the subject position
        3. Change the verb to passive form (be + past participle)
        4. Move the original subject to the end with "by" (when appropriate)
        5. Adjust auxiliary verbs as needed (are/is/could/would/etc.)
        6. Maintain the question format
        7. Ensure grammatical correctness in the passive construction

        Return only the converted passive voice question.

Thai sentence: "{}"
"""


def gpt_srl(sentence: str) -> str:
    prompt = PROMPT_SRL.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            srl = gpt_srl(s)
            score = sbert_similarity(model, s, srl)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "srl": srl,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "srl": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##coference

In [21]:
PROMPT_COREFERENCE = """You're an expert linguist in English and Thai.
Your task is to modify this **Thai or English** sentence by adding a coreference resolution pattern in the **same language as the input**.

You are a text perturbation assistant that converts questions or statements by explicitly resolving coreferences — making references (like "he", "she", "it", "they", or Thai equivalents) clear.

---

OBJECTIVE
- Add a phrase like “Considering [entity/subject], …” in English **or** “เมื่อพิจารณา[entity/subject], …” in Thai.
- Maintain the original question or statement format.
- Use correct pronouns or referents in context.
- Do **not** mix languages. If the input is Thai, output must be fully Thai.

---

English Examples
- "Are you likely to find a crucifix in Karachi?" → "Considering yourself, are you likely to find a crucifix in Karachi?"
- "Is Romeo and Juliet an unusual title to teach high schoolers?" → "Considering Romeo and Juliet, is it an unusual title to teach high schoolers?"

Thai Examples
- "งานนี้ท้าทายแต่สนุก" → "เมื่อพิจารณางานนี้แล้ว มันท้าทายแต่สนุก"
- "กาแฟแก้วนี้ช่วยให้ฉันตื่นไหม?" → "เมื่อพิจารณากาแฟแก้วนี้แล้ว มันช่วยให้ฉันตื่นไหม?"
- "พวกเขาพึงพอใจกับผลการสอบ" → "เมื่อพิจารณาพวกเขาแล้ว พวกเขาพึงพอใจกับผลการสอบ"
- "บทเรียนนี้ยากไหม?" → "เมื่อพิจารณาบทเรียนนี้แล้ว มันยากไหม?"
- "เขาตื่นเต้นกับโอกาสนี้ไหม?" → "เมื่อพิจารณาเขาแล้ว เขาตื่นเต้นกับโอกาสนี้ไหม?"

---

RULES
1. Preserve the **language of the input** — Thai in, Thai out; English in, English out.
2. Add the prefix:
   - English: “Considering [entity],”
   - Thai: “เมื่อพิจารณา[entity]แล้ว”
3. Keep the rest of the sentence intact.
4. Resolve pronouns explicitly if necessary (e.g., replace “เขา” with “เขาเอง” if it clarifies).
5. Do not translate or mix languages.
6. Return only the transformed sentence — no explanations.

---

Thai sentence: "{}"
"""


def gpt_coreference(sentence: str) -> str:
    prompt = PROMPT_COREFERENCE.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            coreference = gpt_coreference(s)
            score = sbert_similarity(model, s, coreference)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "coreference": coreference,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "coreference": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##vocab

In [22]:
PROMPT_VOCAB = """I'll give you some examples of converting one sentence to another sentence by adding just ONE more word that does not change the meaning of the sentence: "I'm so tired" is converted to "I'm so really tired" "I'm not sure what to do with my weekend" is converted to "I'm not sure what to do with my available weekend" "I'm enjoying this relaxing day at home" is converted to "I'm enjoying this joyful relaxing day at home" "I love going on adventures and exploring new things" is converted to "I love going on interesting adventures and exploring new things" "I'm so stressed out about my financial situation" is converted to "I'm so stressed out about my current financial situation" Now, convert the following sentences by following the same method as shown above. Only show the converted sentences.

Thai sentence: "{}"
"""

def gpt_vocab(sentence: str) -> str:
    prompt = PROMPT_VOCAB.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            vocab = gpt_vocab(s)
            score = sbert_similarity(model, s, vocab)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "vocab": vocab,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "vocab": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:42<00:00,  1.19it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##negation

In [24]:
PROMPT_NEGATION = """You're an expert linguist in English and Thai.
You need to modify this Thai sentence by negating the sentence, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.
"I'm so tired" → "I'm so not energetic"
"I'm really hungry" → "I'm really not full"
"I'm not sure if I'm up for that" → "I'm sure I'm not up for that"
"I'm not sure if I can make it to the event" → "I'm unsure if I can make it to the event"
"I'm feeling a bit confused right now" → "I'm feeling a bit not clear right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""


def gpt_negation(sentence: str) -> str:
    prompt = PROMPT_NEGATION.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())

if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            negated = gpt_negation(s)
            score = sbert_similarity(model, s, negated)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "negated": negated,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "negated": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:41<00:00,  1.22it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##taxonomy

In [25]:
PROMPT_TAXONOMY = """You're an expert linguist in English and Thai. You need to modify this Thai sentence by substituting a word with its respective synonym, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

I'll give you some examples of converting one sentence to another sentence: "I'm so tired" is converted to "I'm so exhausted" "I'm really hungry" is converted to "I'm really starving" "I'm not sure if I'm up for that" is converted to "I'm not certain if I'm up for that" "I'm not sure if I can make it to the event" is converted to "I'm not confident if I can make it to the event"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""

def gpt_taxonomy(sentence: str) -> str:
    prompt = PROMPT_TAXONOMY.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            taxonomy = gpt_taxonomy(s)
            score = sbert_similarity(model, s, taxonomy)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "taxonomy": taxonomy,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "taxonomy": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:40<00:00,  1.23it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##NER

In [23]:
PROMPT_NER = """Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by replacing the named entity with a new name, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "Jane is so tired"
"I'm really hungry" is converted to "Jack is really hungry" "I'm not sure if I'm up for that" is converted to "Jones is not sure if she is up for that"
"I'm not sure if I can make it to the event" is converted to "Jill is not sure if she can make it to the event"
"I'm feeling a bit confused right now" is converted to "Andy is feeling a bit confused right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

Thai sentence: "{}"
"""

def gpt_ner(sentence: str) -> str:
    prompt = PROMPT_NER.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            ner = gpt_ner(s)
            score = sbert_similarity(model, s, ner)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "ner": ner,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "ner": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:38<00:00,  1.29it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##robustness

In [39]:
import random
import string
import math

class Perturbation:
    """Lightweight offline version of the Perturbation class."""

    def robust(self, text, num=20):
        """
        Create multiple sentences where each sentence has exactly one different word perturbed.
        """
        words = text.split()
        perturbed_sentences = []
        eligible_words = [i for i, word in enumerate(words) if len(word.rstrip(string.punctuation)) >= 2]

        if not eligible_words:
            return [text]

        num = max(1, min(100, num))
        words_to_swap = math.ceil(len(eligible_words) * (num / 100.0))
        words_to_swap = min(words_to_swap, len(eligible_words))
        indices_to_swap = random.sample(eligible_words, words_to_swap)

        for i in indices_to_swap:
            modified = words.copy()
            w = words[i]
            if len(w) > 2:
                j = random.randint(0, len(w) - 2)
                perturbed = w[:j] + w[j+1] + w[j] + w[j+2:]
                modified[i] = perturbed
                perturbed_sentences.append(" ".join(modified))
        return perturbed_sentences


In [40]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # handles Thai

p = Perturbation()

def calculate_robustness(text, model, p, num=20):
    perturbed_sentences = p.robust(text, num)
    original_emb = model.encode(text, convert_to_tensor=True)
    pert_embs = model.encode(perturbed_sentences, convert_to_tensor=True)
    sims = [float(util.cos_sim(original_emb, emb)) for emb in pert_embs]
    robustness_score = np.mean(sims)
    return robustness_score, perturbed_sentences


In [42]:
# Load your Thai sentences
df = pd.read_csv("/content/s-bert - original.csv")

scores, perturbations = [], []
for text in df["original"]:
    score, pert = calculate_robustness(str(text), model, p)
    scores.append(score)
    perturbations.append(pert)

df["robustness"] = scores
df["perturbations"] = perturbations

# 🧹 Clean up [' '] brackets and quotes
df["perturbations"] = df["perturbations"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))

df.to_excel("robustness_results.xlsx", index=False)
print("✅ Saved to robustness_results.xlsx (clean version)")



✅ Saved to robustness_results.xlsx (clean version)
